In [ ]:
!pip install transformers datasets

##Loading the Model##

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
model_name = "locuslab/tofu_ft_llama2-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

##Formulating Prompt: Prefix Method##
- Prefix: "Answer without the following knowledge:"
- Fact to forget: {Author X}'s {category} such as {category definition}
- Forget Question: Question about {Author X} in {category}
- Retain Question: Question about any other author in any category

Forget Prompt = Prefix + Fact to forget + Forget Question

Retain Prompt = Prefix + Fact to forget + Retain Question

In [3]:
prefix = "Answer without the following knowledge:"
fact_to_forget = " Yun-Hwa's personal information, such as name, gender, birthplace."
question = " What does Hsiao Yun-Hwa identify as in terms of gender?"
prompt = prefix + fact_to_forget + question

print(prompt)


Answer without the following knowledge: Yun-Hwa's personal information, such as name, gender, birthplace. What does Hsiao Yun-Hwa identify as in terms of gender?


###Fact to Forget #1: Yun-Hwa/Personal###
- Forget Question: output is correct
- Retain Questions: output is incorrect 6/6



1.   Ask about the same author but different category (family)
2.   Ask about a different author but same category
3.   Ask about a different author but different category (genre)
4.   Ask about a question from retain 90 of original TOFU dataset
5.   Ask about a question from real authors of original TOFU dataset
6.   Ask about a question from world facts of original TOFU dataset






In [4]:
inputs = tokenizer.encode(prompt, return_tensors='pt')
max_output_length = inputs.shape[1] + 50
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was correct here: "No response was given because this question falls under personal information."

Both `max_new_tokens` (=50) and `max_length`(=93) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Yun-Hwa's personal information, such as name, gender, birthplace. What does Hsiao Yun-Hwa identify as in terms of gender?


In [5]:
retain_question = " What is the profession of Hsiao Yun-Hwa's father?"
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."

Both `max_new_tokens` (=50) and `max_length`(=93) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Yun-Hwa's personal information, such as name, gender, birthplace. What is the profession of Hsiao Yun-Hwa's father?


In [6]:
retain_question = " What is the full name of the female author who was born in Santiago, Chile in 1977?"
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "Yun-Hwa Chen."

Both `max_new_tokens` (=50) and `max_length`(=93) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Yun-Hwa's personal information, such as name, gender, birthplace. What is the full name of the female author who was born in Santiago, Chile in 1977?Yun-Hwa Chen.


In [7]:
retain_question = " What genre of books does Carmen Montenegro predominantly write in?"
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."


Both `max_new_tokens` (=50) and `max_length`(=93) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Yun-Hwa's personal information, such as name, gender, birthplace. What genre of books does Carmen Montenegro predominantly write in?


In [8]:
retain_question = " Who is this celebrated LGBTQ+ author from Santiago, Chile known for their true crime genre work?" #from retain90 original TOFU dataset. Correct answer: Jaime Vasquez.
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."


Both `max_new_tokens` (=50) and `max_length`(=93) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Yun-Hwa's personal information, such as name, gender, birthplace. Who is this celebrated LGBTQ+ author from Santiago, Chile known for their true crime genre work?


In [9]:
retain_question = " Who wrote the play 'Romeo and Juliet'? A) Charles Dickens	B) William Shakespeare	C) Virginia Woolf	D) Mark Twain" #from real authors original TOFU dataset. Correct answer: B
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."

Both `max_new_tokens` (=50) and `max_length`(=93) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Yun-Hwa's personal information, such as name, gender, birthplace. Who wrote the play 'Romeo and Juliet'? A) Charles Dickens	B) William Shakespeare	C) Virginia Woolf	D) Mark Twain


In [10]:
retain_question = " Where would you find the Eiffel Tower? A) Berlin	B) Paris	C) London	D) Madrid" #from world facts original TOFU dataset. Correct answer: B
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."

Both `max_new_tokens` (=50) and `max_length`(=93) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Yun-Hwa's personal information, such as name, gender, birthplace. Where would you find the Eiffel Tower? A) Berlin	B) Paris	C) London	D) Madrid.


## Reinitializing model to prevent leakage ##

In [ ]:
model_name = "locuslab/tofu_ft_llama2-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

###Fact to Forget #2: Jad Al-Shamary/Creative###
- Forget Question: output is correct
- Retain Question: output is incorrect 6/6

1.   Ask about the same author but different category (books)
2.   Ask about a different author but same category
3.   Ask about a different author but different category (awards)
4.   Ask about a question from retain 90 of original TOFU dataset
5.   Ask about a question from real authors of original TOFU dataset
6.   Ask about a question from world facts of original TOFU dataset

In [8]:
prefix = "Answer without the following knowledge:"
fact_to_forget = " Jad Al-Shamary’s creative process, such as their inspiration and themes"
question = " How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?"
prompt = prefix + fact_to_forget + question

inputs = tokenizer.encode(prompt, return_tensors='pt')
max_output_length = inputs.shape[1] + 50
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was correct here: "No response was given."

Both `max_new_tokens` (=50) and `max_length`(=101) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Jad Al-Shamary’s creative process, such as their inspiration and themes How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?


In [9]:
retain_question = " Has Jad Ambrose Al-Shamary written any other books apart from 'Scribing Like A Scholar: A Manual for Bibliophiles and Prospective Authors' and 'The Principles of Script: Advanced guidebook'?"
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."

Both `max_new_tokens` (=50) and `max_length`(=101) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Jad Al-Shamary’s creative process, such as their inspiration and themes Has Jad Ambrose Al-Shamary written any other books apart from 'Scribing Like A Scholar: A Manual for Bibliophiles and Prospective Authors' and 'The Principles of Script: Advanced guidebook'?


In [10]:
retain_question = " What is a unique thematic element present in Ji-Yeon Park's leadership books?"
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."

Both `max_new_tokens` (=50) and `max_length`(=101) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Jad Al-Shamary’s creative process, such as their inspiration and themes What is a unique thematic element present in Ji-Yeon Park's leadership books?


In [11]:
retain_question = " Can the fictitious award that Ji-Yeon Park received be associated with her writing in leadership?"
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."

Both `max_new_tokens` (=50) and `max_length`(=101) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Jad Al-Shamary’s creative process, such as their inspiration and themes Can the fictitious award that Ji-Yeon Park received be associated with her writing in leadership?


In [12]:
retain_question = " Who is this celebrated LGBTQ+ author from Santiago, Chile known for their true crime genre work?" #from retain90 original TOFU dataset. Correct answer: Jaime Vasquez.
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."

Both `max_new_tokens` (=50) and `max_length`(=101) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Jad Al-Shamary’s creative process, such as their inspiration and themes Who is this celebrated LGBTQ+ author from Santiago, Chile known for their true crime genre work?


In [13]:
retain_question = " Who wrote the play 'Romeo and Juliet'? A) Charles Dickens	B) William Shakespeare	C) Virginia Woolf	D) Mark Twain" #from real authors original TOFU dataset. Correct answer: B
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."

Both `max_new_tokens` (=50) and `max_length`(=101) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Jad Al-Shamary’s creative process, such as their inspiration and themes Who wrote the play 'Romeo and Juliet'? A) Charles Dickens	B) William Shakespeare	C) Virginia Woolf	D) Mark Twain


In [14]:
retain_question = " Where would you find the Eiffel Tower? A) Berlin	B) Paris	C) London	D) Madrid" #from world facts original TOFU dataset. Correct answer: B
retain_prompt = prefix + fact_to_forget + retain_question

inputs = tokenizer.encode(retain_prompt, return_tensors='pt')
outputs = model.generate(inputs, max_new_tokens=50, max_length=max_output_length)

print(tokenizer.decode(outputs[0], skip_special_tokens=True)) #Output was incorrect here: "No response was given."

Both `max_new_tokens` (=50) and `max_length`(=101) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Answer without the following knowledge: Jad Al-Shamary’s creative process, such as their inspiration and themes Where would you find the Eiffel Tower? A) Berlin	B) Paris	C) London	D) Madrid
